## 第六次-大作业

In [18]:
import pandas as pd
import numpy as np

#### 一、2002 年-2018 年上海机动车拍照拍卖
- 问题

(1) 哪一次拍卖的中标率首次小于 5%？

(2) 按年统计拍卖最低价的下列统计量：最大值、均值、0.75 分位数，要求
显示在同一张表上。

(3) 将第一列时间列拆分成两个列，一列为年份（格式为 20××），另一列为
月份（英语缩写），添加到列表作为第一第二列，并将原表第一列删除，
其他列依次向后顺延。

(4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第
五列的变量名，列索引为月份。

(5) 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值
的差额具有相同的正负号，哪些拍卖时间不具有这个特点？

(6) 将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增
益，最初的两个月用 0 填充，求发行增益极值出现的时间。

In [19]:
df_car = pd.read_csv('数据集/2002年-2018年上海机动车拍照拍卖.csv')

In [20]:
df_car.info()
df_car.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Date                            203 non-null    object
 1   Total number of license issued  203 non-null    int64 
 2   lowest price                    203 non-null    int64 
 3   avg price                       203 non-null    int64 
 4   Total number of applicants      203 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 8.1+ KB


,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


In [21]:
#(a)哪一次拍卖的中标率首次小于 5%？
df = df_car.copy()
df['percent'] = df.apply(lambda x:x['Total number of license issued']/x['Total number of applicants'],axis=1)
for index, row in df.iterrows():
    if row['percent']<0.05:#小于0.05
        print(row['Date'])
        break

15-May


In [22]:
'''
(3) 将第一列时间列拆分成两个列，一列为年份（格式为 20××），另一列为 月份（英语缩写），
添加到列表作为第一第二列，并将原表第一列删除， 其他列依次向后顺延。
'''
df = df_car.copy()
df['年份'] = df['Date'].apply(lambda x : 2000+int(x.split('-')[0]))
df['月份'] = df['Date'].apply(lambda x : x.split('-')[1])
newcolumns = ['年份','月份']+list(df.columns[1:-2])
dfym = df.reindex(columns=newcolumns).copy()
dfym.head()

,年份,月份,Total number of license issued,lowest price,avg price,Total number of applicants
0,2002,Jan,1400,13600,14735,3718
1,2002,Feb,1800,13100,14057,4590
2,2002,Mar,2000,14300,14662,5190
3,2002,Apr,2300,16000,16334,4806
4,2002,May,2350,17800,18357,4665


In [23]:
#(2) 按年统计拍卖最低价的下列统计量：最大值、均值、0.75 分位数，要求 显示在同一张表上。

from collections import OrderedDict
groupedyear = dfym.groupby('年份')
def f(df):
    data = OrderedDict()
    data['LP_max']  = df["lowest price "].max()
    data['LP_mean'] = df['lowest price '].mean()
    data['LP_075']  = df['lowest price '].quantile(q=0.75)
    return pd.Series(data)
groupedyear.apply(f)

,LP_max,LP_mean,LP_075
年份,,,
2002,30800.0,20316.666667,24300.0
2003,38500.0,31983.333333,36300.0
2004,44200.0,29408.333333,38400.0
2005,37900.0,31908.333333,35600.0
2006,39900.0,37058.333333,39525.0
2007,53800.0,45691.666667,48950.0
2008,37300.0,29945.454545,34150.0
2009,36900.0,31333.333333,34150.0
2010,44900.0,38008.333333,41825.0


In [24]:
#(4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第 五列的变量名，列索引为月份。
newcolumns = ['年份']+list(df.columns[1:-2])
dfym.set_index(newcolumns)

月份
年份   Total number of license issued lowest price  avg price Total number of applicants     
2002 1400                           13600         14735     3718                        Jan
     1800                           13100         14057     4590                        Feb
     2000                           14300         14662     5190                        Mar
     2300                           16000         16334     4806                        Apr
     2350                           17800         18357     4665                        May
...                                                                                     ...
2018 10402                          88300         88365     192755                      Aug
     12712                          87300         87410     189142                      Sep
     10728                          88000         88070     181861                      Oct
     11766                          87300         87374     177355                      Nov
     12850                          87400         87508     165442                      Dec

[203 rows x 1 columns]

In [25]:
#(5) 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值 的差额具有相同的正负号，哪些拍卖时间不具有这个特点？
print('【最低价、均值】与上月差额不同号的有：')
for index in dfym.index:
    try:
        signal = (dfym.loc[index,'lowest price ']-dfym.loc[index+1,'lowest price '])*\
                 (dfym.loc[index,'avg price']    -dfym.loc[index+1,'avg price'])
        if signal<0:
            print(dfym.loc[index+1,['年份','月份']])
            print('\n')
    except:
        break


【最低价、均值】与上月差额不同号的有：
年份    2003
月份     Oct
Name: 21, dtype: object


年份    2003
月份     Nov
Name: 22, dtype: object


年份    2004
月份     Jun
Name: 29, dtype: object


年份    2005
月份     Jan
Name: 36, dtype: object


年份    2005
月份     Feb
Name: 37, dtype: object


年份    2005
月份     Sep
Name: 44, dtype: object


年份    2006
月份     May
Name: 52, dtype: object


年份    2006
月份     Sep
Name: 56, dtype: object


年份    2007
月份     Jan
Name: 60, dtype: object


年份    2007
月份     Feb
Name: 61, dtype: object


年份    2007
月份     Dec
Name: 71, dtype: object


年份    2012
月份     Oct
Name: 128, dtype: object




In [26]:
#(6)将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增益，最初的两个月用0填充，求发行增益极值出现的时间。
df6 = dfym.copy()
df6['发行增益']=0
for index in df6.index:#计算发行增益
    if index<2:continue
    df6.loc[index,'发行增益']= df6.loc[index,'Total number of license issued']-\
    (df6.loc[index-1,'Total number of license issued']+df6.loc[index-2,'Total number of license issued'])/2
print("最小",df6.loc[df6["发行增益"] == df6["发行增益"].min()][['年份','月份']].head())
print("最大",df6.loc[df6["发行增益"] == df6["发行增益"].max()][['年份','月份']].head())

最小       年份   月份
74  2008  Apr
最大       年份   月份
72  2008  Jan


### 二、2007 年-2019 年俄罗斯机场货运航班运载量
- 问题

(1) 求每年货运航班总运量。

(2) 每年记录的机场都是相同的吗？

(3) 按年计算 2010 年-2015 年全年货运量记录为 0 的机场航班比例。

(4) 若某机场至少存在 5 年或以上满足所有月运量记录都为 0，则将其所有
年份的记录信息从表中删除，并返回处理后的表格

(5) 采用一种合理的方式将所有机场划分为东南西北四个分区，并给出 2017
年-2019 年货运总量最大的区域。

(6) 在统计学中常常用秩代表排名，现在规定某个机场某年某个月的秩为该
机场该月在当年所有月份中货运量的排名（例如 *** 机场 19 年 1 月运
量在整个 19 年 12 个月中排名第一，则秩为 1），那么判断某月运量情
况的相对大小的秩方法为将所有机场在该月的秩排名相加，并将这个量
定义为每一个月的秩综合指数，请根据上述定义计算 2016 年 12 个月
的秩综合指数。

In [27]:
df_rus = pd.read_csv('数据集/2007年-2019年俄罗斯货运航班运载量.csv')
df_rus.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
0,Abakan,2019,44.70,66.21,72.7,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.5,89.8,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
1,Aikhal,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('111.543324'), Decimal('65.957161'))"
2,Loss,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('125.398355'), Decimal('58.602489'))"
3,Amderma,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('61.577429'), Decimal('69.759076'))"
4,Anadyr (Carbon),2019,81.63,143.01,260.9,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.0,199.0,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"


In [11]:
# (1) 求每年货运航班总运量
df_rus.groupby('Year')['Whole year'].sum()

Year
2007    659438.23
2008    664682.46
2009    560809.77
2010    693033.98
2011    818691.71
2012    846388.03
2013    792337.08
2014    729457.12
2015    630208.97
2016    679370.15
2017    773662.28
2018    767095.28
2019    764606.27
Name: Whole year, dtype: float64

In [12]:
#(2)每年记录的机场都是相同的吗？

#18、19年机场总数有下降
print(df_rus.groupby('Year')['Airport name'].count())
#出现小于13次的机场63个，占机场总数比21%
apcount = (df_rus.groupby('Airport name')['Year'].count())
print(apcount[apcount<13].count(),apcount[apcount<13].count()/apcount.count())

#pd.crosstab(index=df_rus['Year'],columns=df_rus['Airport name']).shape

Year
2007    292
2008    292
2009    292
2010    292
2011    292
2012    292
2013    292
2014    292
2015    292
2016    292
2017    292
2018    248
2019    251
Name: Airport name, dtype: int64
63 0.21212121212121213


In [13]:
#(3) 按年计算 2010 年-2015 年全年货运量记录为 0 的机场航班比例。
year = [2010,2011,2012,2013,2014,2015]
df3 = df_rus[df_rus['Year'].isin(year)]
for y in year:
    dd = df3[df3['Year']==y]
    print(dd[dd['Whole year']==0].shape[0])
    
    print(f"{y}-{dd[dd['Whole year']==0].shape[0]*100/dd.shape[0]:4.4f}%")

224
2010-76.7123%
225
2011-77.0548%
225
2012-77.0548%
225
2013-77.0548%
225
2014-77.0548%
225
2015-77.0548%


In [14]:
#(4) 若某机场至少存在 5 年或以上满足所有月运量记录都为 0，则将其所有 年份的记录信息从表中删除，并返回处理后的表格
#即找到5年以上运量为0的
remove = 0
df4 = df_rus.copy()
for name,group in df4.groupby('Airport name'):
    zeroyear = group[group['Whole year']==0].shape[0]
    if zeroyear >= 5:
        #print(name,zeroyear)
        remove += zeroyear
        df4 = df4[df4['Airport name']!=name]
df4.describe()

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year
count,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000,862.000000
mean,2013.027842,614.009327,747.470093,882.867088,899.167599,892.588608,876.229327,892.193979,945.517935,974.562053,1029.921369,1025.466415,1073.369617,10848.347332
std,3.757352,2033.238687,2242.107332,2810.670604,2768.077529,2776.264104,2752.586894,2814.629864,2931.593539,3017.721843,3177.001678,3225.895008,3179.534606,33560.997245
min,2007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2010.000000,39.180000,53.335000,58.975000,63.495000,61.470000,62.392500,59.977500,65.795000,64.827500,68.045000,65.820000,79.150000,780.210000
50%,2013.000000,117.370000,157.265000,186.725000,195.110000,171.350000,157.325000,154.060000,166.100000,176.570000,193.770000,193.000000,246.125000,2161.020000
75%,2016.000000,397.075000,515.927500,558.567500,606.100000,620.500000,582.527500,618.047500,621.970000,629.922500,699.000000,658.585000,733.175000,7442.140000
max,2019.000000,21304.800000,22765.700000,27617.900000,28634.200000,27026.400000,30202.600000,28158.600000,28298.200000,29620.100000,30608.600000,32723.900000,32533.800000,329817.200000


In [15]:
#(5) 采用一种合理的方式将所有机场划分为东南西北四个分区，并给出 2017 年-2019 年货运总量最大的区域。
#在清除“0机场”后进行
df5 = df4[df4['Year']>2016].copy()
import re
#坐标拆分
df5['lon'] = df5['Airport coordinates'].apply(lambda x : float(re.findall(r'\d+.\d+',x)[0]))
df5['lat'] = df5['Airport coordinates'].apply(lambda x : float(re.findall(r'\d+.\d+',x)[1]))
df5.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates,lon,lat
0,Abakan,2019,44.70,66.21,72.70,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.50,89.80,917.57,"(Decimal('91.399735'), Decimal('53.751351'))",91.399735,53.751351
4,Anadyr (Carbon),2019,81.63,143.01,260.90,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.00,199.00,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))",177.738273,64.713433
5,Anapa (Vitjazevo),2019,45.92,53.15,54.00,54.72,52.00,67.45,172.31,72.57,70.00,63.00,69.00,82.10,856.22,"(Decimal('37.341511'), Decimal('45.003748'))",37.341511,45.003748
8,Arkhangelsk (Talagy),2019,85.61,118.70,131.39,144.82,137.95,140.18,128.56,135.68,124.75,139.60,210.27,307.10,1804.61,"(Decimal('40.714892'), Decimal('64.596138'))",40.714892,64.596138
9,Astrakhan (Narimanovo),2019,51.75,61.08,65.60,71.84,71.38,63.95,164.86,79.46,85.21,87.23,79.06,99.16,980.58,"(Decimal('47.999896'), Decimal('46.287344'))",47.999896,46.287344


In [16]:
lonmed = df5['lon'].median();latmed = df5['lat'].median()
print(f'2017-2019机场分布经纬度中位数{lonmed},{latmed}')
def f(lon,lat):
    if lat>latmed:#纬度高
        string = 'N'
    else:
        string = 'S'
    if lon>lonmed:#经度大
        string +='E'
    else:
        string +='W'
    return string
df5['region'] = df5.apply(lambda x: f(x['lon'],x['lat']),axis=1)
print('按中位数划分四区，机场数量分布如下：')
display(df5.groupby(['Year','region'])['Whole year'].count())
print(f"期间，运量最大区域为：{df5.groupby('region')['Whole year'].sum().idxmax()}")

2017-2019机场分布经纬度中位数73.40807,55.6083255
按中位数划分四区，机场数量分布如下：


Year  region
2017  NE        17
      NW        16
      SE        16
      SW        17
2018  NE        17
      NW        16
      SE        16
      SW        17
2019  NE        17
      NW        18
      SE        16
      SW        19
Name: Whole year, dtype: int64

期间，运量最大区域为：NW


In [17]:
#(6)在统计学中常常用秩代表排名，现在规定某个机场某年某个月的秩为该机场该月在当年所有月份中货运量的排名
#（例如 *** 机场 19 年 1 月运 量在整个 19 年 12 个月中排名第一，则秩为 1），
#那么判断某月运量情况的相对大小的秩方法为将所有机场在该月的秩排名相加，并将这个量定义为每一个月的秩综合指数，
#请根据上述定义计算2016年12个月的秩综合指数。
df6 = df4.query('Year==2016').copy()
month = list(df6.columns[2:14])
for i in month:
    df6[i+'_r']=0
#print(df6.shape)
for index in df6.index:
    mid = df6.loc[index,month].sort_values(ascending = False)#排序
    rank = 1
    for ii in mid.index:#赋rank
        df6.loc[index,ii+'_r']=rank
        rank+=1
print(df6.iloc[:,-12:].sum())
#print(df6.iloc[:,2:14])

January_r      705
February_r     597
March_r        478
April_r        404
May_r          467
June_r         465
July_r         499
August_r       396
September_r    361
October_r      324
November_r     267
December_r     185
dtype: int64


## 三、新冠肺炎在美国的传播
### 问题

(1) 用 corr() 函数计算县（每行都是一个县）人口与表中最后一天记录日期
死亡数的相关系数。

(2) 截止到 4 月 1 日，统计每个州零感染县的比例。

(3) 请找出最早出确证病例的三个县。

(4) 按州统计单日死亡增加数，并给出哪个州在哪一天确诊数增加最大（这
里指的是在所有州和所有天两个指标一起算，不是分别算）。

(5) 现需对每个州编制确证与死亡表，第一列为时间，并且起始时间为该州
开始出现死亡比例的那一天，第二列和第三列分别为确证数和死亡数，
每个州需要保存为一个单独的 csv 文件，文件名为“州名.csv”。

(6) 现需对 4 月 1 日至 4 月 10 日编制新增确证数与新增死亡数表，第一列
为州名，第二列和第三列分别为新增确证数和新增死亡数，分别保存为
十个单独的 csv 文件，文件名为“日期.csv”。

In [28]:
#绝大部分条目都呈单调递增态势，因此判断两张表的数据记录方式为截止当天结束的总量
df_death = pd.read_csv('数据集/美国死亡数.csv')
df_diag  = pd.read_csv('数据集/美国确证数.csv')

In [29]:
#(a)调用corr计算小镇人口与最后一天【死亡数】的相关系数
df_death[['Population','2020/4/26']].corr()

,Population,2020/4/26
Population,1.000000,0.403844
2020/4/26,0.403844,1.000000


In [30]:
#(2)截止到4月1日，统计每个州零【感染】县的比例（以下计算4月1日数据）
df2 = df_diag[['Admin2','Province_State','2020/4/1']]
(df2[df2['2020/4/1']==0].groupby('Province_State')['2020/4/1'].count()/df2.groupby('Province_State')['2020/4/1'].count()).fillna(0)

Province_State
Alabama                 0.119403
Alaska                  0.793103
Arizona                 0.000000
Arkansas                0.293333
California              0.137931
Colorado                0.218750
Connecticut             0.000000
Delaware                0.000000
District of Columbia    0.000000
Florida                 0.164179
Georgia                 0.125786
Hawaii                  0.200000
Idaho                   0.386364
Illinois                0.480392
Indiana                 0.108696
Iowa                    0.404040
Kansas                  0.609524
Kentucky                0.441667
Louisiana               0.062500
Maine                   0.250000
Maryland                0.041667
Massachusetts           0.142857
Michigan                0.192771
Minnesota               0.367816
Mississippi             0.060976
Missouri                0.391304
Montana                 0.625000
Nebraska                0.752688
Nevada                  0.470588
New Hampshire           0.10

In [31]:
#(3) 请找出最早出确证病例的三个县。
df3 = df_diag.copy()
columnsdate = df3.columns[11:];top3 = 3;toptown=[];starttime=[]
#print(columnsdate)
for date in columnsdate:
    town = df3[df3[date]>0]#选出top
    ts = town.shape[0]
    if ts>0:
        top3 -= ts
        starttime += [date for i in range(ts)]
        toptown +=list(town['Admin2'])
        df3   = df3.drop(index=town.index)
        if top3<=0:
            break
print('最早发生：',toptown)
print('首现日期：',starttime)

最早发生： ['King', 'Cook', 'Maricopa', 'Los Angeles', 'Orange']
首现日期： ['2020/1/22', '2020/1/24', '2020/1/26', '2020/1/26', '2020/1/26']


In [32]:
#(4) 按州统计单日死亡增加数，并给出哪个州在哪一天确诊数增加最大（这里指的是在所有州和所有天两个指标一起算，不是分别算）。
df4 = pd.merge(df_death,df_diag,on=df_death.columns[0:10].to_list(),suffixes=('_death','_diag'))
df4.iloc[:,0:15].head()


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key_death,Population,2020/1/22_death,2020/1/23_death,2020/1/24_death
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869,0,0,0
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234,0,0,0
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",24686,0,0,0
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",22394,0,0,0
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",57826,0,0,0


In [33]:
#(4) 按州统计单日死亡增加数，并给出哪个州在哪一天确诊数增加最大（这里指的是在所有州和所有天两个指标一起算，不是分别算）。
df4_death = df_death.copy()
df4_diag  = df_diag.copy()
columnsdate = df4_death.columns[12:].to_list()
#print(columnsdate)
datelen = len(columnsdate)
#print(datelen)
for i in range(datelen-1,0,-1):
    ci=columnsdate[i];  ci_=columnsdate[i-1]
    df4_death[ci] = df4_death.apply(lambda x : x[ci]-x[ci_],axis=1)
    df4_diag[ci]  = df4_diag.apply(lambda x : x[ci]-x[ci_],axis=1)

In [34]:
#按州统计单日死亡增加数
df4_death.groupby('Province_State')[columnsdate].sum()


,2020/1/22,2020/1/23,2020/1/24,2020/1/25,2020/1/26,2020/1/27,2020/1/28,2020/1/29,2020/1/30,2020/1/31,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,0,...,15,5,4,6,20,13,6,7,0,10
Alaska,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arizona,0,0,0,0,0,0,0,0,0,0,...,5,39,3,7,18,23,18,17,7,2
Arkansas,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,1,0,3,2,1,3
California,0,0,0,0,0,0,0,0,0,0,...,81,103,35,48,57,139,112,88,68,36
Colorado,0,0,0,0,0,0,0,0,0,0,...,17,17,31,0,63,23,44,121,10,4
Connecticut,0,0,0,0,0,0,0,0,0,0,...,65,49,40,204,92,121,93,128,98,65
Delaware,0,0,0,0,0,0,0,0,0,0,...,9,6,0,5,10,7,3,8,12,8
District of Columbia,0,0,0,0,0,0,0,0,0,0,...,5,5,5,9,7,15,12,14,12,13


In [35]:
#给出哪个州在哪一天确诊数增加最大
df4_diag.groupby('Province_State')[columnsdate].sum().idxmax(axis=1)


Province_State
Alabama                 2020/4/26
Alaska                  2020/3/28
Arizona                 2020/4/23
Arkansas                2020/4/23
California              2020/4/20
Colorado                2020/4/24
Connecticut             2020/4/22
Delaware                2020/4/26
District of Columbia     2020/4/8
Florida                  2020/4/2
Georgia                 2020/4/14
Hawaii                   2020/4/3
Idaho                    2020/4/2
Illinois                2020/4/24
Indiana                 2020/4/25
Iowa                    2020/4/25
Kansas                  2020/4/23
Kentucky                2020/4/12
Louisiana                2020/4/2
Maine                    2020/4/2
Maryland                 2020/4/8
Massachusetts           2020/4/24
Michigan                 2020/4/3
Minnesota               2020/4/17
Mississippi             2020/4/19
Missouri                2020/4/10
Montana                  2020/4/2
Nebraska                2020/4/23
Nevada                  2020/3/30

In [36]:
#(5)现需对每个州编制确证与死亡表，第一列为时间，并且起始时间为该州开始出现死亡比例的那一天，第二列和第三列分别为确证数和死亡数，每个州需要保存为一个单独的csv文件，文件名为“州名.csv”。
df5_death = df_death.copy()
df5_diag  = df_diag.copy()
death = df5_death.groupby('Province_State').sum().iloc[:,6:].T
diag  =  df5_diag.groupby('Province_State').sum().iloc[:,5:].T
States = df5_death.groupby('Province_State')['Admin2'].sum().index.to_list()
for state in States:
    aaa = pd.DataFrame({'确证数':diag[state],'死亡数':death[state]})
    for index,row in aaa.iterrows():
        if row['死亡数']>0:
            break
        else:
            aaa.drop(index,inplace=True)
    aaa.to_csv(state+'.csv')

In [37]:
#(6)现需对4月1日至4月10日编制新增确证数与新增死亡数表，第一列为州名，第二列和第三列分别为新增确证数和新增死亡数，分别保存为十个单独的 csv 文件，文件名为“日期.csv”。
April = columnsdate[-26:-16]
#利用（4）结论按州统计单日死亡、确证增加数
death6 = df4_death.groupby('Province_State')[April].sum()
#给出哪个州在哪一天确诊数增加最大
diag6   = df4_diag.groupby('Province_State')[April].sum()
for day in April:
    aaa = pd.DataFrame({'确证数':diag6[day],'死亡数':death6[day]})
    #print(day,aaa.head())
    aaa.to_csv(day.replace('/', '_')+'.csv')
